In [24]:
import sys
sys.path.insert(0, '../..')
from util import carregamento_dados as cd, ferramentas_grafos as fg
import pandas as pd
from collections import defaultdict

In [25]:
dump_path = '../../pickles/licitacoes/'
relacoes_entre_cnpjs = cd.salvar_relacoes_entre_cnpjs()
informacoes_licitacoes = cd.salvar_informacoes_licitacoes()
cnpjs_por_licitacao = cd.salvar_cnpjs_por_licitacao()
%time


CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.58 µs


In [26]:
cliques = pd.read_pickle('cliques_picles')
cliques.head(1)

,ano,municipio,tipo_processo_licitatorio,id_licitacao,valor,vinculo_em_uso,quantidade_cnpjs,quantidade_vinculos,densidade_grafo,tam_clique_encontrada,lista_de_cnpjs_compondo_clique
0,2016,Abaeté,Tomada de Preços,985585,235160.08,1,4,1,0.166667,2,19268374000110;14054958000170;


In [27]:
d_relacoes = cd.cnpjs_relacionados_por_cnpj(relacoes_entre_cnpjs)
d_licitacoes = cd.cnpjs_por_licitacao(cnpjs_por_licitacao)
%time

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.34 µs


In [28]:
dados_licitacao = cnpjs_por_licitacao.values
licitacoes = [licitacao for licitacao, _ in dados_licitacao]
%time

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 3.58 µs


In [29]:
d = {
    licitacao: {
        'cnpjs' : [],
        'ano' : None,
        'municipio': None,
        'modalidade': None,
        'valor': None,
        'grafo': None,
        'cliques': None
    } for licitacao in licitacoes
}

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs


In [30]:
for licitacao, cnpj in cnpjs_por_licitacao.values:
    d[licitacao]['cnpjs'].append(cnpj)

%time
    

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.1 µs


In [31]:
for l in informacoes_licitacoes.values:
    try:
        d[l[0]]['municipio'] = l[1]
        d[l[0]]['ano'] = l[5]
        d[l[0]]['modalidade'] = l[3]
        d[l[0]]['valor'] = l[7]
    except:
        # Exceção para licitações que não possuem informações de CNPJs licitantes
        pass

%time

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.05 µs


In [32]:
for licitacao in licitacoes:
    grafo = fg.gera_grafo_licitacao(
        licitacao, d_relacoes, d_licitacoes
    )
    cliques = fg.lista_cliques(grafo)

    d[licitacao]['grafo'] = grafo
    d[licitacao]['cliques'] = cliques

%time

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [33]:
# c : cliques dictionary
c = defaultdict(dict)

clique_id = 0
for licitacao in pd.Series(licitacoes).unique():
    for clique in d[licitacao]['cliques']:
        tamanho_clique = len(clique)
        if tamanho_clique >= 2:
            printable_cnpjs = ''
            for cnpj in clique:
                printable_cnpjs += (str(cnpj) + ';')
            c[clique_id] = {
                'ano' : d[licitacao]['ano'],
                'municipio': d[licitacao]['municipio'],
                'tipo_processo_licitatorio': d[licitacao]['modalidade'],
                'id_licitacao' : licitacao,
                'valor': d[licitacao]['valor'],
                'vinculo_em_uso': '1',
                'quantidade_cnpjs': len(d[licitacao]['cnpjs']),
                'quantidade_vinculos': fg.calcula_quantidade_arestas(d[licitacao]['grafo']),
                'densidade_grafo': fg.calcula_densidade(d[licitacao]['grafo']),
                'tam_clique_encontrada': tamanho_clique,
                'lista_de_cnpjs_compondo_clique' : printable_cnpjs,
                'cnpjs': clique
            }
            clique_id += 1

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


Vamos verificar em quantas cliques cada CNPJ aparece.
Primeiro precisamos levantar todos os cnpjs (reduzimos o universo consultando somente CNPJs em cliques)

In [34]:
CNPJs_em_cliques = set()
for id in range(clique_id):
    for cnpj in c[id]['cnpjs']:
        CNPJs_em_cliques.add(cnpj)

In [43]:
cnpjs_cliques = {
    cnpj: {
        'qtdade_licitacoes_que_figurou_com_alguem_com_vinculo': 0,
        'lista_licitacoes_onde_isso_ocorreu': ''
    } for cnpj in CNPJs_em_cliques
}

In [44]:
for id in range(clique_id):
    for cnpj in c[id]['cnpjs']:
        cnpjs_cliques[cnpj]['qtdade_licitacoes_que_figurou_com_alguem_com_vinculo'] += 1
        cnpjs_cliques[cnpj]['lista_licitacoes_onde_isso_ocorreu'] += c[id]['id_licitacao'] + ';'

In [45]:
cnpjs_cliques = pd.DataFrame.from_dict(
    data=cnpjs_cliques,
    orient='index'
)

In [46]:
cnpjs_cliques.to_csv('relatorio3.csv')

In [47]:
cnpjs_cliques

,qtdade_licitacoes_que_figurou_com_alguem_com_vinculo,lista_licitacoes_onde_isso_ocorreu
12420164000319,7,876321;876330;876370;952879;953158;948555;953320;
04830177000176,2,765742;765682;
26092483000195,1,977286;
71189278000105,1,962116;
11718529000161,2,892107;892209;
...,...,...
01783016000180,1,983890;
10769908000118,1,899381;
17370034000170,1,829084;
11656957000107,1,863178;
